In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('../../')))
import logging
# Logging setup
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

module_logger = logging.getLogger()
module_logger.addHandler(handler)
module_logger.setLevel(logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.base import LanguageModel, Openai, Chain, Prompt, ProbabilisticComponent, SymbolicComponent, Threads, Concept, ConceptRegistry, use_registry, RegistryAccessor
import pandas as pd
import dtale

# tests

In [4]:
# pd.DataFrame(chain.memory)
# dtale.show(pd.DataFrame(lm3.memory))

# parsers
# list as output
# routers
# retriever


In [5]:
# add placeholders and import 


chain1 = Chain([
                ProbabilisticComponent(LanguageModel("OpenAI1"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google1"), Prompt("what do you think of this joke: {joke}?"), 'other'), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), Prompt("can you explain why this joke is funny: {joke}?"), 'subject')])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'lala')])

threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {else}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'kaka')])


# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

## Component

In [6]:
component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), 'joke')
initial_concepts = [Concept(name='other', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    component.run()

In [7]:
component.memory

['Response of OpenAI2: to (tell me a joke about computer.)']

## Chain

In [8]:
# Run the chain
initial_concepts = [Concept(name='something', content='something'), Concept(name='subject', content='computer')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    out = chain1.run()

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:200: UserWarning:

Overwrite Warning: 
{
    "content": "computer",
    "inputted": true,
    "level": 0,
    "link": null,
    "name": "subject"
}

 Is being overwritten by:
 {
    "content": "Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)",
    "inputted": false,
    "level": 3,
    "link": null,
    "name": "subject"
}



In [9]:
concept_registry

{
    "concepts": {
        "joke": {
            "content": "Response of OpenAI1: to (tell me a joke about computer.)",
            "inputted": false,
            "level": 1,
            "link": null,
            "name": "joke"
        },
        "other": {
            "content": "Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)",
            "inputted": false,
            "level": 2,
            "link": null,
            "name": "other"
        },
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "something"
        },
        "subject": {
            "content": "Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)",
            "inputted": false,
            "level": 3,
            "link": null,
            "name": "subject"
        }
  

## Chain of chains

In [10]:
chain3 = Chain([chain1, chain2])

In [11]:

initial_concepts = [Concept(name='subject', content='computer'), Concept(name='else', content='else')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain3.run()
concept_registry

{
    "concepts": {
        "else": {
            "content": "else",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "else"
        },
        "joke": {
            "content": "Response of OpenAI2: to (tell me a joke about computer.)",
            "inputted": false,
            "level": 1,
            "link": null,
            "name": "joke"
        },
        "lala": {
            "content": "Response of Microsoft2: to (can you explain why this joke is funny: Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about computer.)?)?)",
            "inputted": false,
            "level": 3,
            "link": null,
            "name": "lala"
        },
        "subject": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "subject"
        },
        "think": {
            "content": "Response of Google2: to 

## Threads

In [12]:
initial_concepts = [Concept(name='subject', content='computer'), Concept(name='lala', content='something'), 
                        Concept(name='think', content='something'), Concept(name='else', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    threads1._run()


/Users/fuad/Documents/projects/lexflow/src/base/executable.py:200: UserWarning:

Overwrite Warning: 
{
    "content": "something",
    "inputted": true,
    "level": 0,
    "link": null,
    "name": "think"
}

 Is being overwritten by:
 {
    "content": "Response of Google2: to (what do you think of this joke: something?)",
    "inputted": false,
    "level": 1,
    "link": null,
    "name": "think"
}



In [13]:
concept_registry

{
    "concepts": {
        "else": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "else"
        },
        "joke": {
            "content": "Response of OpenAI2: to (tell me a joke about something.)",
            "inputted": false,
            "level": 1,
            "link": null,
            "name": "joke"
        },
        "kaka": {
            "content": "Response of Microsoft2: to (can you explain why this joke is funny: something?)",
            "inputted": false,
            "level": 1,
            "link": null,
            "name": "kaka"
        },
        "lala": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "lala"
        },
        "subject": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "subject"
      

# More complex

In [14]:
# add placeholders and import 
threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {something}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'kaka')])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'lala'),
                threads1
               ])

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), 
                          Prompt("tell me a joke about {subject}."), 'haha'), 
                ProbabilisticComponent(LanguageModel("Google1"), 
                          Prompt("what do you think of this joke: {haha}?"), 'other'), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {haha}?"), 'subject'),
                chain2
               ])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), 'joke')



In [15]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

chain3 = Chain([chain1, chain2, threads1])
initial_concepts = [Concept(name='subject', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:200: UserWarning:

Overwrite Warning: 
{
    "content": "Response of OpenAI2: to (tell me a joke about Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?).)",
    "inputted": false,
    "level": 4,
    "link": null,
    "name": "joke"
}

 Is being overwritten by:
 {
    "content": "Response of OpenAI2: to (tell me a joke about Response of Microsoft2: to (can you explain why this joke is funny: Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?).)?)?).)",
    "inputted": false,
    "level": 7,
    "link": null,
    "name": "joke"
}

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:200: UserWarning:

Overwrite Warning: 
{
    "content": "Response of Google2: to (wha

In [16]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

chain3 = Chain([chain1, chain2, threads1])
initial_concepts = [Concept(name='subject', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

In [17]:
concept_registry

{
    "concepts": {
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "something"
        },
        "subject": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "subject"
        }
    },
    "initial_concepts": [
        {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "subject"
        },
        {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "something"
        }
    ]
}

In [18]:
# object names should be unique otherwise they will be overwritten 

# Listify

In [19]:
# add placeholders and import 
threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {something}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'kaka')])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'lala'),
                threads1
               ])

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), 
                          Prompt("tell me a joke about {subject}."), 'haha'), 
                ProbabilisticComponent(LanguageModel("Google1"), 
                          Prompt("what do you think of this joke: {haha}?"), 'other'), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {haha}?"), 'subject'),
                chain2
               ])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), 'joke')

chain3 = Chain([chain1, chain2, threads1])


In [20]:
# concept_registry

In [21]:
initial_concepts = [Concept(name='other', content='computer\nelez'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
# 
# maybe change the functions such that it outputs a tuple of name and value.
chain = Chain([
    SymbolicComponent('test',
                      function=lambda x: x['other'].content.split('\n'),
                      # function=lambda x: [x['other'].content + 'processed'], 
                      input_concept_names=['other'],
                      # output_concept_names=['other_1']
                     )
])
with use_registry(concept_registry):
    chain.run()

In [22]:
# todo split . 
# create an iterator outputter that names concepts
# produce an iterator of values and a corresponding iterator of names 

In [23]:
concept_registry

{
    "concepts": {
        "other": {
            "content": "computer\nelez",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "other"
        },
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "something"
        },
        "test_0": {
            "content": "computer",
            "inputted": false,
            "level": 1,
            "link": {
                "other": {
                    "content": "computer\nelez",
                    "inputted": true,
                    "level": 0,
                    "link": null,
                    "name": "other"
                }
            },
            "name": "test_0"
        },
        "test_1": {
            "content": "elez",
            "inputted": false,
            "level": 1,
            "link": {
                "other": {
                    "content": "computer\ne

In [25]:

    
chain1 = Chain(
    [SymbolicComponent('test',
                      function=lambda x: x['other'].content.split('\n'),
                      input_concept_names=['other'],
                      # output_concept_names=['other_1']
                     ), 
     Threads(ProbabilisticComponent.component_iterator(model=LanguageModel('TestModel'),
                                                      prompt=Prompt('tell me a joke about {}'), 
                                                      concept_pattern=r'^test_'))]
)

initial_concepts = [Concept(name='other', content='computer\nelez'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

In [26]:
concept_registry

{
    "concepts": {
        "other": {
            "content": "computer\nelez",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "other"
        },
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "link": null,
            "name": "something"
        },
        "test_0": {
            "content": "computer",
            "inputted": false,
            "level": 1,
            "link": {
                "other": {
                    "content": "computer\nelez",
                    "inputted": true,
                    "level": 0,
                    "link": null,
                    "name": "other"
                }
            },
            "name": "test_0"
        },
        "test_0_output": {
            "content": "Response of TestModel: to (tell me a joke about test_0)",
            "inputted": false,
            "level": 2,
            "link": null,
        

# Test live LLM

In [29]:
import json

format_string = """Strictly output in JSON format. Take a deep breath. The JSON should have the following format:
[{"step": "..."}, {"step": "..."}]"""

def test_function(x):
    one = x['joke'].content
    print(one)
    two = json.loads(one)#['jokes']
    print(two)
    return [e['joke'] for e in two]

chain = Chain([ProbabilisticComponent(model=Openai('real'), 
                                      prompt=Prompt(template='Give me a 5 step plan of steps to answer the question {other}',
                                                    output_format=format_string), 
                                      output_concept_name='joke'),
               SymbolicComponent(
                   name='symbolic_joke',
                      function=test_function,
                      input_concept_names=['joke'],
                      # output_concept_names=['other_1']
                     ),
               Threads(ProbabilisticComponent.component_iterator(model=Openai('TestModel'),
                                                                 prompt=Prompt('{}. Do you find this joke funny'), 
                                                                 concept_pattern=r'^symbolic_joke_'))
              ])

initial_concepts = [Concept(name='other', content='how do you find the truth'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    o = chain.run()
    

Step 1: Ensure that you have the necessary data or information to generate the desired JSON output. This may include accessing a database, API, or any other source that contains the required data.

Step 2: Determine the programming language or tool you will use to process the data and generate the JSON output. Common options include Python, JavaScript, or any other language with JSON support.

Step 3: Write code to retrieve the necessary data and store it in a suitable data structure. This could involve querying a database, making API requests, or parsing files.

Step 4: Create a loop or iteration process to iterate through the data and generate the JSON output. For each item in the data, create a dictionary or object with the required format and append it to a list or array.

Step 5: Finally, convert the list or array containing the generated dictionaries or objects into a JSON string using the appropriate function or method provided by your chosen programming language or tool. Ensure

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
concept_registry

In [ ]:
# reoragnise api for different components make it consistant. 
# make input and output names clear 
# structured output for open ai . 
# structured output for other models


# retry 
# retry based on ... should put the loads step in a symbolic and make the retry based on teh symbolic step 
# retry with error message
# number of retries 
# 

# add a retriever
# add memory 


# meta data creator of documents
# multi step retireval plan by probabilistic component


# tool creator / agents. compare to autogen
# tool box
# it can create tools as functions and then add it to the box


In [ ]:
o

# Data Class experiment

In [30]:
from pydantic.dataclasses import dataclass as pydantic_dataclass

In [26]:
@pydantic_dataclass
class Test:
    something = []

In [31]:
t = Test(e='that')

In [34]:
t

Test()

In [30]:
schema = {
  "type": "object",
  "properties": {
    "ingredients": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "unit": { 
            "type": "string",
            "enum": ["grams", "ml", "cups", "pieces", "teaspoons"]
          },
          "amount": { "type": "number" }
        },
        "required": ["name", "unit", "amount"]
      }
    },
    "instructions": {
      "type": "array",
      "description": "Steps to prepare the recipe (no numbering)",
      "items": { "type": "string" }
    },
    "time_to_cook": {
      "type": "number",
      "description": "Total time to prepare the recipe in minutes"
    }
  },
  "required": ["ingredients", "instructions", "time_to_cook"]
}

In [4]:
schema = {
  "type": "object",
  "properties": {
    "ingredients": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "unit": { 
            "type": "string",
            "enum": ["grams", "ml", "cups", "pieces", "teaspoons"]
          },
          "amount": {
            "oneOf": [
              { "type": "number" },
              { "type": "string", "enum": ["small", "medium", "large"] }
            ]
          }
        },
        "required": ["name", "unit", "amount"]
      }
    },
    "instructions": {
      "type": "array",
      "description": "Steps to prepare the recipe",
      "items": { "type": "string" }
    },
    "time_to_cook": {
      "type": "number",
      "description": "Total time to prepare the recipe in minutes"
    }
  },
  "required": ["ingredients", "instructions", "time_to_cook"]
}


In [5]:
schema = {
  "type": "object",
  "properties": {
    "ingredients": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "unit": { 
            "type": "string",
            "enum": ["grams", "ml", "cups", "pieces", "teaspoons"]
          },
          "amount": {
            "oneOf": [
              { "type": "number" },
              { "type": "string", "enum": ["small", "medium", "large"] }
            ]
          }
        },
        "required": ["name", "unit", "amount"]
      }
    },
    "instructions": {
      "type": "array",
      "description": "Steps to prepare the recipe",
      "items": { "type": "string" }
    },
    "time_to_cook": {
      "type": "number",
      "description": "Total time to prepare the recipe in minutes"
    }
  },
  "required": ["ingredients", "instructions", "time_to_cook"]
}

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Provide a recipe for spaghetti bolognese"}
  ],
  functions=[{"name": "set_recipe", "parameters": schema}],
  function_call={"name": "set_recipe"},
  temperature=0,
)

print(completion.choices[0].message.function_call.arguments)

{
  "ingredients": [
    {
      "name": "spaghetti",
      "unit": "grams",
      "amount": 400
    },
    {
      "name": "ground beef",
      "unit": "grams",
      "amount": 500
    },
    {
      "name": "onion",
      "unit": "pieces",
      "amount": 1
    },
    {
      "name": "garlic cloves",
      "unit": "pieces",
      "amount": 2
    },
    {
      "name": "carrot",
      "unit": "pieces",
      "amount": 1
    },
    {
      "name": "celery stalk",
      "unit": "pieces",
      "amount": 1
    },
    {
      "name": "canned tomatoes",
      "unit": "grams",
      "amount": 400
    },
    {
      "name": "red wine",
      "unit": "ml",
      "amount": 125
    },
    {
      "name": "olive oil",
      "unit": "teaspoons",
      "amount": 2
    },
    {
      "name": "salt",
      "unit": "teaspoons",
      "amount": 1
    },
    {
      "name": "pepper",
      "unit": "teaspoons",
      "amount": 1
    }
  ],
  "instructions": [
    "Heat the olive oil in a large pan over 

In [32]:
{
  "ingredients":[
    {"name": "spaghetti", "unit": "grams", "amount": 400},
    {"name": "ground beef", "unit": "grams", "amount": 500},
    {"name": "onion", "unit": "pieces", "amount": 1},
    {"name": "garlic cloves", "unit": "pieces", "amount": 2},
    {"name": "carrot", "unit": "pieces", "amount": 1},
    {"name": "celery stalk", "unit": "pieces", "amount": 1},
    {"name": "canned tomatoes", "unit": "grams", "amount": 400},
    {"name": "red wine", "unit": "ml", "amount": 125},
    {"name": "olive oil", "unit": "ml", "amount": 30},
    {"name": "salt", "unit": "teaspoons", "amount": 1},
    {"name": "pepper", "unit": "teaspoons", "amount": 1}
  ],
  "instructions": [
    "Heat the olive oil in a large pan over medium heat. Add the finely chopped onion, carrot, celery, and minced garlic and cook until softened.",
    "Add the ground beef to the pan and cook until browned.",
    "Pour in the red wine and let it simmer until the alcohol has evaporated.",
    "Add the canned tomatoes, salt, and pepper. Reduce the heat to low, cover the pan, and let it simmer for about 1 hour, stirring occasionally.",
    "In the meantime, cook the spaghetti in a large pot of boiling salted water according to the package instructions until al dente.",
    "Drain the spaghetti and add it to the pan with the Bolognese sauce. Toss well to combine.",
    "Serve the Spaghetti Bolognese with a sprinkle of freshly grated Parmesan cheese on top."
  ],
  "time_to_cook": 90
}

{'ingredients': [{'name': 'spaghetti', 'unit': 'grams', 'amount': 400},
  {'name': 'ground beef', 'unit': 'grams', 'amount': 500},
  {'name': 'onion', 'unit': 'pieces', 'amount': 1},
  {'name': 'garlic cloves', 'unit': 'pieces', 'amount': 2},
  {'name': 'carrot', 'unit': 'pieces', 'amount': 1},
  {'name': 'celery stalk', 'unit': 'pieces', 'amount': 1},
  {'name': 'canned tomatoes', 'unit': 'grams', 'amount': 400},
  {'name': 'red wine', 'unit': 'ml', 'amount': 125},
  {'name': 'olive oil', 'unit': 'ml', 'amount': 30},
  {'name': 'salt', 'unit': 'teaspoons', 'amount': 1},
  {'name': 'pepper', 'unit': 'teaspoons', 'amount': 1}],
 'instructions': ['Heat the olive oil in a large pan over medium heat. Add the finely chopped onion, carrot, celery, and minced garlic and cook until softened.',
  'Add the ground beef to the pan and cook until browned.',
  'Pour in the red wine and let it simmer until the alcohol has evaporated.',
  'Add the canned tomatoes, salt, and pepper. Reduce the heat to 